In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
from sklearn import preprocessing

In [2]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')
unscaled_inputs = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:, -1]

### Preprocessing

In [3]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal = np.delete(unscaled_inputs, indices_to_remove, axis = 0)
targets_equal = np.delete (targets_all, indices_to_remove, axis=0)

In [4]:
# Standardize the inputs

scaled_inputs = preprocessing.scale(unscaled_inputs_equal)

In [5]:
# shuffle

shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal[shuffled_indices]

In [6]:
# split into train, validation, test

samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1773.0 3579 0.4953897736797988
238.0 447 0.5324384787472036
226.0 448 0.5044642857142857


In [7]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Loading data

In [8]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model

In [30]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2)

model.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets), 
          verbose = 2 
          )  

Epoch 1/100
36/36 - 0s - loss: 0.6040 - accuracy: 0.6871 - val_loss: 0.5250 - val_accuracy: 0.7472 - 306ms/epoch - 8ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4793 - accuracy: 0.7631 - val_loss: 0.4500 - val_accuracy: 0.7427 - 38ms/epoch - 1ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4202 - accuracy: 0.7782 - val_loss: 0.4112 - val_accuracy: 0.7517 - 37ms/epoch - 1ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3861 - accuracy: 0.7885 - val_loss: 0.3876 - val_accuracy: 0.7875 - 38ms/epoch - 1ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3680 - accuracy: 0.7994 - val_loss: 0.3732 - val_accuracy: 0.8009 - 36ms/epoch - 997us/step
Epoch 6/100
36/36 - 0s - loss: 0.3564 - accuracy: 0.7983 - val_loss: 0.3649 - val_accuracy: 0.7987 - 39ms/epoch - 1ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3508 - accuracy: 0.8055 - val_loss: 0.3585 - val_accuracy: 0.8031 - 38ms/epoch - 1ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3430 - accuracy: 0.8097 - val_loss: 0.3522 - val_accuracy: 0.8098 - 37ms/epoch - 1ms/step
Epoch 9/100
3

### Test the model

In [31]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 839us/step - loss: 0.3778 - accuracy: 0.7723
